In [ ]:
import boto3
import json
import uuid

In [ ]:
lambda_client = boto3.client("lambda")
s3 = boto3.client("s3")

In [ ]:
session = boto3.session.Session()
sts = session.client("sts")
# 현재 계정 ID 가져오기
account_id = sts.get_caller_identity()["Account"]
# 현재 리전 가져오기
region = session.region_name

In [ ]:
S3_BUCKET = f"bedrock-mm-{account_id}-{region}"
S3_KEY_TASK_TEMPLATE = "tasks/{task_id}/upload/{file_name}"

LAMBDA_FUN_NAME_NOVA_MME_START_TASK = "bedrock-mm-nova-srv-start-task"
LAMBDA_FUN_NAME_NOVA_MME_DELETE_TASK = "bedrock-mm-nova-srv-delete-video-task"
NOVA_MME_MODEL_ID = "amazon.nova-2-multimodal-embeddings-v1:0"

## 비디오 작업 호출 - Nova MME

In [ ]:
local_path = "./samples/content-moderation-demo.mp4"
file_name = local_path.split('/')[-1]

# task_id 생성
task_id = str(uuid.uuid4())

# 대상 s3 키 구성
s3_key = S3_KEY_TASK_TEMPLATE.format(task_id=task_id, file_name=file_name)

# 요청에 필요한 기타 값
task_name = file_name # 작업 이름 설정, 기본값은 파일 이름 사용
request_by = "workshop-user" # 요청 사용자 이름 설정

In [ ]:
s3.upload_file(local_path, S3_BUCKET, s3_key)

In [ ]:
# MME 작업 호출
request = {
          "ModelId": NOVA_MME_MODEL_ID,
          "EmbedMode": "AUDIO_VIDEO_COMBINED",
          "DurationS": 5,
          "TaskId": task_id,
          "FileName": file_name,
          "TaskName": file_name,
          "Video": {
            "S3Object": {
              "Bucket": S3_BUCKET,
              "Key": s3_key
            }
          },
          "RequestBy": request_by
        }

In [ ]:
response = lambda_client.invoke(
    FunctionName=LAMBDA_FUN_NAME_NOVA_MME_START_TASK,
    InvocationType="RequestResponse",  # async
    Payload=json.dumps(request),
)

# Lambda 응답 파싱
response_payload = response["Payload"].read()

In [ ]:
response_payload

## 정리
delete-task 엔드포인트를 호출하여 업로드된 비디오(S3), S3에 생성된 모든 메타데이터 파일, DynamoDB에 저장된 메타데이터를 포함한 모든 작업 관련 상태를 제거합니다. MME 작업의 경우 S3 벡터 인덱스에서 비디오 관련 벡터도 삭제됩니다.

모든 비디오와 작업을 삭제하려면 다음 코드의 주석을 해제하세요.

In [ ]:
'''
response = lambda_client.invoke(
    FunctionName=LAMBDA_FUN_NAME_NOVA_MME_DELETE_TASK,
    InvocationType="RequestResponse",  # async
    Payload=json.dumps({"TaskId": task_id}),
)

# Lambda 응답 파싱
response_payload = response["Payload"].read()
response_payload
'''